# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [163]:
!pip freeze | grep gensim

gensim==4.2.0


In [164]:
!pip freeze | grep tensorflow

tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-macos==2.10.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [165]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [166]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [167]:
wv['bull']

array([-6.36810884e-02,  3.45573239e-02, -3.47808190e-02,  6.20321520e-02,
        2.17961224e-05, -9.66805741e-02, -6.47831988e-03,  1.39668867e-01,
       -8.60485360e-02, -4.89086062e-02, -2.92837862e-02, -1.07435465e-01,
       -4.76226676e-03,  4.27819751e-02, -5.14860498e-03, -6.73625246e-02,
        4.02188487e-02, -4.33906801e-02,  1.27968956e-02, -8.12143162e-02,
        4.51644212e-02,  1.94133837e-02,  4.31880131e-02, -1.96067840e-02,
        7.99922552e-03, -9.42261051e-03, -5.06271683e-02, -1.17065641e-03,
       -3.16557102e-02, -4.33510263e-03,  6.05767183e-02,  8.86101928e-03,
        9.06662736e-03, -9.13012400e-02, -2.42706630e-02,  3.98919731e-02,
        3.37548964e-02, -3.73601168e-02, -3.02217752e-02, -9.86185148e-02,
       -3.59206796e-02, -9.91641954e-02, -5.51460944e-02,  3.39386202e-02,
        7.91284963e-02,  1.89216039e-03, -3.80051583e-02, -3.74957435e-02,
        6.30161092e-02,  4.76687700e-02,  3.03818621e-02, -6.71535879e-02,
       -5.13387956e-02,  

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [168]:
len(word2vec.wv['dog'])

100

In [169]:
wv.vector_size

100

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [170]:
word2vec.wv.most_similar('dog', topn=10)
#seems not really well done

[('passion', 0.9904700517654419),
 ('bill', 0.9899911880493164),
 ('ugly', 0.9878901839256287),
 ('calls', 0.9873920679092407),
 ('birth', 0.9873619675636292),
 ('killing', 0.9867575764656067),
 ('rush', 0.9865365624427795),
 ('wear', 0.986534059047699),
 ('beetle', 0.9865320920944214),
 ('costume', 0.9864662289619446)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [171]:
v_bull = word2vec.wv['bull']
v_bear = word2vec.wv['bear']

v_result = v_bull-v_bear

In [172]:
word2vec.wv.similar_by_vector(v_result)

[('are', -0.07561937719583511),
 ('were', -0.11933178454637527),
 ('is', -0.15362685918807983),
 ('many', -0.1641428917646408),
 ('no', -0.17255032062530518),
 ('resisted', -0.1762220561504364),
 ("it's", -0.17665985226631165),
 ('has', -0.18262918293476105),
 ('was', -0.19374150037765503),
 ('acting', -0.19730433821678162)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [173]:
v_good = word2vec.wv['good']
v_bad = word2vec.wv['bad']

v_result = v_good - v_bad
#v_result

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [174]:
v_stupid = word2vec.wv['stupid']

res = v_good - v_bad + v_stupid
#res

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [175]:
word2vec.wv.similar_by_vector(res)

[('nice', 0.766713559627533),
 ('good', 0.7417517304420471),
 ('great', 0.7368097305297852),
 ('also', 0.7172247171401978),
 ('decent', 0.7127443552017212),
 ('biased', 0.6988264322280884),
 ('potential', 0.6979751586914062),
 ('deed', 0.6953942775726318),
 ('considered', 0.6940988302230835),
 ('done', 0.6932544112205505)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [176]:
v_boy = word2vec.wv['boy']
v_girl = word2vec.wv['girl']
v_man = word2vec.wv['man']
v_woman = word2vec.wv['woman']

res = v_boy - v_girl + v_woman
word2vec.wv.similar_by_vector(res)

[('boy', 0.9598641991615295),
 ('career', 0.9430646300315857),
 ('conscience', 0.9419369101524353),
 ('lover', 0.938258171081543),
 ('friend', 0.9378591775894165),
 ('sister', 0.9370297193527222),
 ('vigilante', 0.9354276657104492),
 ('child', 0.9353976249694824),
 ('secretary', 0.9336825013160706),
 ('ira', 0.9331446290016174)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [177]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=300)

In [178]:
len(word2vec_2.wv['fog'])

300

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [179]:
word2vec_2.wv.key_to_index

{'the': 0,
 'a': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'i': 8,
 'it': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'but': 16,
 'movie': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'one': 25,
 'be': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'they': 31,
 'an': 32,
 'so': 33,
 'like': 34,
 'who': 35,
 'from': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'if': 40,
 'out': 41,
 'about': 42,
 "it's": 43,
 'has': 44,
 'what': 45,
 'some': 46,
 'there': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'no': 52,
 'up': 53,
 'she': 54,
 'my': 55,
 'time': 56,
 'even': 57,
 'which': 58,
 'would': 59,
 'really': 60,
 'only': 61,
 'had': 62,
 'story': 63,
 'me': 64,
 'see': 65,
 'can': 66,
 'their': 67,
 'were': 68,
 'well': 69,
 'than': 70,
 'much': 71,
 'get': 72,
 'do': 73,
 'great': 74,
 'been': 75,
 'we': 76,
 'first': 77,
 'bad': 78,
 'because': 79,
 'into': 80,
 'other': 81,


In [180]:
len(X_train[4])

81

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [181]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=300,min_count=7)
word2vec_4 = Word2Vec(sentences=X_train, vector_size=300,min_count=3)

In [182]:
len(word2vec_3.wv.key_to_index), len(word2vec_4.wv.key_to_index)

(6064, 12116)

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [183]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=300,min_count=3,window=9)
len(word2vec.wv.key_to_index)

8006

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [184]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    
    embedding_size = word2vec.vector_size
    word_embeddings = []
    for word in sentence:
        if word in word2vec.wv.key_to_index:
            word_embedding = word2vec.wv[word]
            word_embeddings.append(word_embedding)
    if len(word_embeddings) > 0:
        sentence_embedding = np.array(word_embeddings)
    else:
        sentence_embedding = np.zeros(shape=(1,embedding_size))
    return sentence_embedding


In [185]:
embedded_sentence = embed_sentence(word2vec, example)
embedded_sentence.shape

(8, 100)

In [186]:
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

In [187]:
embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [188]:
def embedding(word2vec, sentences):
    sentence_embeddings = []
    for i, sentence in enumerate(sentences):
        sentence_embeddings.append(embed_sentence(word2vec,sentence))
    return sentence_embeddings

In [189]:
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [191]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=0)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=0)
assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
